In [30]:
import pandas as pd

In [31]:
df = pd.read_csv('data/amazon_reviews_us_Musical_Instruments_v1_00.tsv', sep='\t', error_bad_lines=False)
df['positive'] = df.star_rating.apply(lambda rating: 1 if rating > 3 else 0)
df.head()

b'Skipping line 5782: expected 15 fields, saw 22\nSkipping line 31979: expected 15 fields, saw 22\nSkipping line 32408: expected 15 fields, saw 22\nSkipping line 45709: expected 15 fields, saw 22\nSkipping line 64585: expected 15 fields, saw 22\n'
b'Skipping line 115742: expected 15 fields, saw 22\nSkipping line 125537: expected 15 fields, saw 22\nSkipping line 127580: expected 15 fields, saw 22\nSkipping line 128696: expected 15 fields, saw 22\n'
b'Skipping line 143851: expected 15 fields, saw 22\nSkipping line 147069: expected 15 fields, saw 22\nSkipping line 149233: expected 15 fields, saw 22\nSkipping line 167536: expected 15 fields, saw 22\nSkipping line 168535: expected 15 fields, saw 22\nSkipping line 171659: expected 15 fields, saw 22\nSkipping line 184533: expected 15 fields, saw 22\nSkipping line 184624: expected 15 fields, saw 22\nSkipping line 184908: expected 15 fields, saw 22\n'
b'Skipping line 219150: expected 15 fields, saw 22\nSkipping line 223178: expected 15 fields, 

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,positive
0,US,45610553,RMDCHWD0Y5OZ9,B00HH62VB6,618218723,AGPtek® 10 Isolated Output 9V 12V 18V Guitar P...,Musical Instruments,3,0,1,N,N,Three Stars,"Works very good, but induces ALOT of noise.",2015-08-31,0
1,US,14640079,RZSL0BALIYUNU,B003LRN53I,986692292,Sennheiser HD203 Closed-Back DJ Headphones,Musical Instruments,5,0,0,N,Y,Five Stars,Nice headphones at a reasonable price.,2015-08-31,1
2,US,6111003,RIZR67JKUDBI0,B0006VMBHI,603261968,AudioQuest LP record clean brush,Musical Instruments,3,0,1,N,Y,Three Stars,removes dust. does not clean,2015-08-31,0
3,US,1546619,R27HL570VNL85F,B002B55TRG,575084461,Hohner Inc. 560BX-BF Special Twenty Harmonica,Musical Instruments,5,0,0,N,Y,I purchase these for a friend in return for pl...,I purchase these for a friend in return for pl...,2015-08-31,1
4,US,12222213,R34EBU9QDWJ1GD,B00N1YPXW2,165236328,Blue Yeti USB Microphone - Blackout Edition,Musical Instruments,5,0,0,N,Y,Five Stars,This is an awesome mic!,2015-08-31,1


In [32]:
df.positive

0         0
1         1
2         0
3         1
4         1
         ..
903999    1
904000    1
904001    1
904002    1
904003    1
Name: positive, Length: 904004, dtype: int64

In [33]:
df.groupby(df.customer_id).size().mean()

1.5782606038915126

In [36]:
from sklearn.model_selection import train_test_split
numerical_features = df[['customer_id', 'product_parent', 'positive']]
numerical_features.drop_duplicates()

X_train, X_test, y_train, y_test = train_test_split(numerical_features.drop('positive', axis=1), numerical_features.positive)

X_train.head()

,customer_id,product_parent
156601,15185554,753647223
852326,16228310,273886806
516377,51790687,900064685
661322,11513157,44258853
78240,40347763,637297052


In [37]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [38]:
clf.score(X_test, y_test)

0.7794169052349326

In [39]:
from sklearn.naive_bayes import MultinomialNB
bayes = MultinomialNB()
bayes.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [40]:
bayes.score(X_test, y_test)

0.5022809633585692

In [41]:
from sklearn.neighbors import KNeighborsClassifier
maxK, maxAcc = 0, 0
for i in range(1, 50):
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(X_train, y_train)
    acc = neigh.score(X_test, y_test)
    if acc > maxAcc:
        maxK, maxAcc = i, acc
    print("k: {} Acc: {}".format(i, acc))
print('Max k:', maxK, 'acc:', maxAcc)

k: 1 Acc: 0.700802208839784
k: 2 Acc: 0.6113557019659205
k: 3 Acc: 0.7538063990867298
k: 4 Acc: 0.7124879978407175
k: 5 Acc: 0.7770806323865823
k: 6 Acc: 0.7549789602700873
k: 7 Acc: 0.7894301352648883
k: 8 Acc: 0.7769257658151955
k: 9 Acc: 0.7968062088220849
k: 10 Acc: 0.7892885429710488
k: 11 Acc: 0.80095220817607
k: 12 Acc: 0.7967442621935301
k: 13 Acc: 0.8035052942243618
k: 14 Acc: 0.8005849531639241
k: 15 Acc: 0.8048459962566538
k: 16 Acc: 0.8029610488449166
k: 17 Acc: 0.805739797611515
k: 18 Acc: 0.8045097145587852
k: 19 Acc: 0.8066114751704638
k: 20 Acc: 0.8055849310401282
k: 21 Acc: 0.8068681112030478
k: 22 Acc: 0.8063105915460551
k: 23 Acc: 0.8072619147702886
k: 24 Acc: 0.8068548369255003
k: 25 Acc: 0.8074167813416755
k: 26 Acc: 0.8071026234397193
k: 27 Acc: 0.8075672231538799
k: 28 Acc: 0.8072707642886536
k: 29 Acc: 0.807779611594639
k: 30 Acc: 0.8075672231538799
k: 31 Acc: 0.8079433276843908
k: 32 Acc: 0.8078282839456462
k: 33 Acc: 0.8080273981088579
k: 34 Acc: 0.80788580581

In [42]:
neigh.score(X_test, y_test)

0.8082353617904345